In [1]:
import psycopg2
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time 
import pandas as pd 
from selenium.common.exceptions import StaleElementReferenceException
import psycopg2

def connect_db() :
    from sqlalchemy import create_engine

    postgres_address = 'localhost'
    postgress_port = '5432'
    postgres_username = 'postgres'
    postgres_password = 'juhari123'
    postgres_database = 'data_trademap'
    postgres = ('postgresql://{username}:{password}@{address}:{port}/{dbname}'.format(username=postgres_username, \
                                                                                      password=postgres_password,
                                                                                      address=postgres_address,
                                                                                      port=postgress_port,
                                                                                      dbname=postgres_database))

    # untuk mengkoneksikan ke database yang digunakan
    con = create_engine(postgres)
    return con 

def get_code_country(connect_db ):
    con = connect_db()
    df = pd.read_sql_query('select * from code_negara', con )

    kode_negara = {}
    for i in range( len(df)) :
        kode_negara[df.code[i]] = df.nama[i]
      
    kombinasi_negara = df.code[df.code!='360']

    return con, kode_negara, kombinasi_negara


def get_key(val, kode_negara):
    for key, value in kode_negara.items():
         if val == value:
             return key
 
    return "key doesn't exist"
 
# Driver Code
 
 

 

In [12]:
con, kode_negara, kombinasi_negara =  get_code_country(connect_db)

In [37]:

def html_source_browser(negara,partner, browser2):
    browser = browser2
    

    # select = Select(WebDriverWait(browser, 20).until(
    #     EC.element_to_be_clickable((By.XPATH, "//select[@id='ctl00_NavigationControl_DropDownList_Country']"))))
    # select.select_by_value(negara)
    # browser.implicitly_wait(20)

    # time.sleep(3)
    
    select = Select(WebDriverWait(browser,30).until(
        EC.element_to_be_clickable((By.XPATH, "//select[@id='ctl00_NavigationControl_DropDownList_Partner']"))))
    select.select_by_value(partner)
    browser.implicitly_wait(20)
    

    time.sleep(5)





 
    html_source = []
    html_source.append((browser.page_source))
    
    
    while True:

        try:
            WebDriverWait(browser,20).until(EC.element_to_be_clickable((By.XPATH,
                                                                         "//table[@id='ctl00_PageContent_MyGridView1']//table/tbody/tr//td/span//following::td[1]/a"))) 
            browser.find_element_by_xpath("//table[@id='ctl00_PageContent_MyGridView1']//table/tbody/tr//td/span//following::td[1]/a").click()
            browser.implicitly_wait(20)
            
            time.sleep(10)
            html_source.append((browser.page_source))
        except TimeoutException:
            break

 
    return html_source

def data_import_export_extract(html_source, angka ):
    soup = BeautifulSoup(html_source,'html.parser')
    global tdTags

    data = soup.findAll('td',{'class':['tabContent']})

    web = []
    for item in data:
        tdTags = item.find_all("tr")
    for item in tdTags:
        for asii in item.find_all("td") :
                #print(no , asii.text)
                #product_code.append(asii.text.strip())
            web.append(asii.text.strip())
    if angka != 1:
        web = web[15:]
    else :
        web = web[35:]


    return web

def insert_into_list(web):
    m = len(web)
    n = 20

    product_code = []
    product_label = []
    value_2016 = []
    value_2017 = []
    value_2018 = []
    value_2019 = []
    value_2020 = []

    for i in range(0, m, n):
        # print(web[i])
        product_code.append(web[i])
        product_label.append(web[i + 1])
        value_2016.append(web[i + 2])
        value_2017.append(web[i + 3])
        value_2018.append(web[i + 4])
        value_2019.append(web[i + 5])
        value_2020.append(web[i + 6])

    # make dataframe
    kolom = ['product_code', 'product_label', '2016', '2017', '2018', '2019', '2020']
    df = pd.DataFrame(columns=kolom)
    df['product_code'] = product_code
    df['product_label'] = product_label
    df['2016'] = value_2016
    df['2017'] = value_2017
    df['2018'] = value_2018
    df['2019'] = value_2019
    df['2020'] = value_2020
    return df



In [47]:


# options = FirefoxOptions()
# options.add_argument("--headless")
browser = webdriver.Chrome('/home/hira/chromedriver'  )
executor_url = browser.command_executor._url
session_id = browser.session_id
browser.get(f"https://www.trademap.org/Bilateral_TS.aspx?nvpm=1|360||004||TOTAL|||2|1|1|2|2|1|1|1|1|1")


select = Select(WebDriverWait(browser, 30).until(EC.element_to_be_clickable(
    (By.XPATH, "//select[@id='ctl00_PageContent_GridViewPanelControl_DropDownList_NumTimePeriod']"))))
select.select_by_value('5')
browser.implicitly_wait(20)

time.sleep(5)

select = Select(WebDriverWait(browser, 30).until(EC.element_to_be_clickable(
    (By.XPATH, "//select[@id='ctl00_NavigationControl_DropDownList_ProductClusterLevel']"))))
select.select_by_value('4')
browser.implicitly_wait(20)


time.sleep(5)

select = Select(WebDriverWait(browser, 30).until(EC.element_to_be_clickable(
    (By.XPATH, "//select[@id='ctl00_PageContent_GridViewPanelControl_DropDownList_PageSize']"))))
select.select_by_value('300')
browser.implicitly_wait(20)


print (session_id)
print (executor_url)

def create_driver_session(session_id, executor_url):
    from selenium.webdriver.remote.webdriver import WebDriver as RemoteWebDriver

    # Save the original function, so we can revert our patch
    org_command_execute = RemoteWebDriver.execute

    def new_command_execute(self, command, params=None):
        if command == "newSession":
            # Mock the response
            return {'success': 0, 'value': None, 'sessionId': session_id}
        else:
            return org_command_execute(self, command, params)

    # Patch the function before creating the driver object
    RemoteWebDriver.execute = new_command_execute

    new_browser = webdriver.Remote(command_executor=executor_url, desired_capabilities={})
    new_browser.session_id = session_id

    # Replace the patched function with original function
    RemoteWebDriver.execute = org_command_execute

    return new_browser

browser2 = create_driver_session(session_id, executor_url)






b2fb663d7122998a00b40a842862ff72
http://127.0.0.1:45183


In [48]:
con, kode_negara, kombinasi_negara =  get_code_country(connect_db)

df =   pd.read_sql_query("""

select code, nama  from  code_negara where  nama not in (
select  importer from trademap_export_fix group by  importer having  count(*) =1225)

	""",con )

wrong = [ x for x in df.code]
for i,kombinasi in enumerate(wrong,1):
    kolom = ['product_code', 'product_label', '2016', '2017', '2018', '2019', '2020','importer','exporter']
    data_exportir = pd.DataFrame(columns=kolom)

    if  i>= 0 and  i <= len(wrong)    :
        print(f"{i}/{len(wrong)} Started")
        partner = kombinasi
        nama_negara = kode_negara['360']
        nama_partner = kode_negara[partner]
        html_source =  html_source_browser('360', partner, browser2)
        for i, data in  enumerate(html_source,1):
            web = data_import_export_extract(data,i)
            df = insert_into_list(web)
            df['importer'] = nama_partner  
            df['exporter'] = nama_negara
            df = df[df.product_code != "1" ]
            df = df[~df.duplicated()]
            data_exportir = pd.concat([df, data_exportir], axis = 0)
        if len (data_exportir) == 1225:
            data_exportir.to_sql('trademap_export_raw_sisa', con=con, if_exists='append')
        else:
            data_exportir.to_sql('trademap_wrongs', con=con, if_exists='append')






1/20 Started
2/20 Started
3/20 Started
4/20 Started
5/20 Started
6/20 Started
7/20 Started
8/20 Started
9/20 Started
10/20 Started
11/20 Started
12/20 Started
13/20 Started
14/20 Started
15/20 Started
16/20 Started
17/20 Started
18/20 Started
19/20 Started
20/20 Started


In [8]:
df = pd.read_sql_query("""

select code from code_negara where  nama in (
select  importer from trademap_wrongs group by  importer
)
	""",con )

df = [ x for x in df.code]

In [9]:
df

['052',
 '112',
 '056',
 '084',
 '204',
 '060',
 '064',
 '074',
 '080',
 '192',
 '208',
 '214',
 '226',
 '231',
 '568',
 '238',
 '838',
 '288',
 '300',
 '304',
 '324',
 '340',
 '360',
 '381',
 '388',
 '440',
 '442',
 '454',
 '458',
 '470',
 '584',
 '480',
 '175',
 '583',
 '662']